# GPT-3 Text Embedding and Semantic Search

This notebook demonstrates how to utilize the power of GPT-3 for text embedding and perform semantic search to find the most similar documents based on a user query.

The GPT-3 language model is a state-of-the-art deep learning model developed by OpenAI. It has been trained on a vast amount of text data and can generate high-quality text responses.

In this notebook, we will perform the following steps:

1. Embedding Texts with GPT-3: We will use GPT-3 to embed a collection of texts into vector representations. These embeddings capture the semantic meaning of the texts.

2. Chunking Texts: If the text documents are larger than the maximum token length supported by GPT-3, we will split them into smaller chunks for efficient processing.

3. Semantic Search: Given a user query, we will calculate the similarity between the query embedding and the embeddings of the text chunks. We will identify and rank the most similar documents based on their similarity scores.

4. Display Results: We will display the top-ranked documents that are most relevant to the user's query.

Let's get started!

In [9]:
import pandas as pd
import openai
import pinecone
from dotenv import load_dotenv
import dataframe_image as dfi
import os
from processor import Preprocessor, TextEmbedder, QueryEngine

# Embedding model. Currently recommended version. 
EMBEDDING_MODEL = 'text-embedding-ada-002'
CHAT_COMPLETION_CTX_LENGTH = 4096
# Define the query length in order to subtract from the maximum length the chunk can contain. 
QUERY_LENGTH = 200
EMBEDDING_ENCODING = 'cl100k_base'
# Number of k most similar docs to return when searching.
K_DOCS = 5

# load the env file from the root directory
load_dotenv('../.env')
openai.api_key=os.environ.get('OPENAI_API_KEY')

In [10]:
# Necessary variables
docx_file = 'data/FiBL-Bericht_Basiskonzept-KErn-final_en_preprocessed.docx'  
# Location where the txt format of the report should be saved. 
txt_file = 'data/sustainability_report.txt'
chunked_txt_dir = 'data/report_chunks'
# location of the csv file to store the embeddings.
embeddings_file = 'data/embeddings.csv'
# embedding length for embedding model 'data/embeddings.csv'
embedding_length = 1536

### Step 1: Preprocesing Step
This step is used to preprocess the docx format of the Sustainability check report. Run it if the docx document needs to be preprocessed.
Currently, the document has already been preprocessed and the data is included in the /data directory. 

In [11]:
preprocessor = Preprocessor(docx_file=docx_file)
preprocessor.preprocess()

Preprocessing of data/FiBL-Bericht_Basiskonzept-KErn-final_en_preprocessed.docx initiated...
Preprocessing done.


### Step 2: Chunking the document
In this step, the document is chunked into texts that fit the size limit specified. The document is chunked if it is too large. 
The chunks are saved under /data/report_chunks.

In [12]:
embedder = TextEmbedder(txt_file=txt_file, output_dir=chunked_txt_dir)

# Setting the chunk size to be 1/2*K, where k is the number of the k most similar docs to use in the query.
# The reason is because the max length is equal to query + knowledge base + response. 
# This number can be changed, but the overall max length has to remain less than the max length, 
chunk_size = int(CHAT_COMPLETION_CTX_LENGTH/(2*K_DOCS))

# Chunk the text.
chunks = embedder.chunk_text(EMBEDDING_ENCODING, chunk_size)


Chunk 0 is of length: 409

Chunk 1 is of length: 409

Chunk 2 is of length: 409

Chunk 3 is of length: 409

Chunk 4 is of length: 409

Chunk 5 is of length: 409

Chunk 6 is of length: 409

Chunk 7 is of length: 409

Chunk 8 is of length: 409

Chunk 9 is of length: 409

Chunk 10 is of length: 409

Chunk 11 is of length: 409

Chunk 12 is of length: 409

Chunk 13 is of length: 409

Chunk 14 is of length: 409

Chunk 15 is of length: 409

Chunk 16 is of length: 409

Chunk 17 is of length: 409

Chunk 18 is of length: 409

Chunk 19 is of length: 409

Chunk 20 is of length: 409

Chunk 21 is of length: 409

Chunk 22 is of length: 409

Chunk 23 is of length: 409

Chunk 24 is of length: 409

Chunk 25 is of length: 409

Chunk 26 is of length: 409

Chunk 27 is of length: 409

Chunk 28 is of length: 409

Chunk 29 is of length: 409

Chunk 30 is of length: 409

Chunk 31 is of length: 409

Chunk 32 is of length: 409

Chunk 33 is of length: 409

Chunk 34 is of length: 409

Chunk 35 is of length: 409

Ch

### Step 3: Embed the chunks
Here, we embed the chunks and create a csv that contains the textual representation and the embeddings. 
The embeddings can be found under: data/embeddings.csv. The embeddings for the current snippets already exist, so embedding again is not necessary.

In [13]:
# embed the chunks and save them in the csv.
embedder.embed_chunks(csv_embeddings_file=embeddings_file)

### Search the text using the embeddings

In [14]:
df = pd.read_csv(embeddings_file)
queryEngine = QueryEngine()

# User query example
query = 'What are the values used in the sustainability score?'
res = queryEngine.search_chunks(csv_embeddings_file=embeddings_file, query=query, k=K_DOCS, pprint=True)
# Results dataframe
res

text

embedding

similarity



,text,embedding,similarity
9,land use\n\nwater use\n\nuse fossil fuels\n\nR...,"[0.02241666428744793, -0.014131942763924599, 0...",0.856460
27,from soy)\n\nregionality\n\n\nThe following e...,"[0.02397286333143711, -0.0025806042831391096, ...",0.837199
34,other.\n\nThe SDGs encompass three dimensions...,"[0.009694411419332027, 0.005684871692210436, 0...",0.829371
1,model that considers the various aspects of s...,"[0.022131573408842087, 8.800480281934142e-05, ...",0.828416
13,"production on methods, but also differences b...","[0.02353695034980774, -0.021789297461509705, -...",0.827208


### (OPTIONAL) Create PNG Of DF result
Create a png of the dataframe for the slides. 

In [150]:
# Creat a PNG image of the dataframe
dfi.export(res, 'images/similarity_table_3NN.png')

### Step 4: Query the knowledge base
Here, we call the query method to query the knowledge base.

In [15]:
max_num_sentences = 2
max_num_words = 20
gpt_response = queryEngine.query(res, query, max_num_sentences, max_num_words)
print(gpt_response)

text = gpt_response.content
print(text)

The concatenated knowledge base: 
 land use

water use

use fossil fuels

Raw material consumption: minerals and metals

The rating scale contains five stages from A until E A Groceries with the Evaluation A has one low low impact on the environment and a product rated E has a high impact on the environment. In addition, plus or minus points can be awarded through certain other criteria. be collected. For example, with sustainability labels such as "Demeter" and "Bio" plus points or through one not sustainable Packaging minuses to be collected.

the fact that life cycle analyzes are used as a basis causes criticism . life cycle analysis According to one accusation, people prefer products from intensive agriculture. In addition the imported Groceries at the Eco score not worse away as local produced (food news, 2021).

Besides that recorded the Eco score subjects How Biodiversity, animal welfare or Mission from crop protection teln (PSM) not directly. However, these aspects are increasi

#### Alternative: Use Pinecone
Instead of using a csv to store the embeddings, you can also use a Pinecone instance.

In [16]:
# DB index
index_name = 'report-knowledge-base'

# initialize connection to pinecone
pinecone.init(
    api_key=os.environ.get("PINECONE_API_KEY"),
    environment="asia-southeast1-gcp-free"
)

# check if index exists
if index_name not in pinecone.list_indexes():
    # create db index if it does not exist. 
    pinecone.create_index(
        index_name,
        dimension=embedding_length,
        metric='cosine',
        metadata_config={'indexed': ['text']}
    )

# connect to index
index = pinecone.Index(index_name)
# View index stats
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 42}},
 'total_vector_count': 42}

In [17]:
#### Save the embeddings in the vector database.
embedder.embed_chunks(vector_db=index)

In [18]:
query_embedding = openai.embeddings_utils.get_embedding(
            query,
            engine=EMBEDDING_MODEL
    )

# retrieve from pinecone
print(query_embedding)
# get relevant contexts
res = index.query(query_embedding, top_k=5, include_metaada=True)
res

[0.018435949459671974, -0.019007818773388863, 0.021894393488764763, -0.0015709362924098969, -0.029437610879540443, 0.01775515452027321, -0.023650845512747765, -0.010845077224075794, -0.015998700633645058, -0.029192524030804634, 0.019307369366288185, 0.01652972213923931, 0.0021240829955786467, -0.008040198124945164, 0.009013736620545387, -0.0006986666703596711, 0.014637109823524952, -0.01631186716258526, 0.013752074912190437, -0.01312574278563261, -0.012737689539790154, 0.0009497101418673992, -0.023732541128993034, 0.0017836850602179766, -0.004163066390901804, 0.016284635290503502, 0.017156053334474564, -0.009660492651164532, 0.01789131388068199, -0.005575717426836491, 0.031207678839564323, -0.006746686529368162, -0.02241179719567299, 0.02103658951818943, -0.0240048598498106, 0.02279304340481758, -0.008237629197537899, 0.0058242082595825195, 0.022588804364204407, -0.01621655561029911, -0.005466790404170752, 0.019225673750042915, -0.003941807895898819, -0.0075432173907756805, -0.00127904

{'matches': [{'id': 'report_chunk_17.txt', 'score': 0.856459677, 'values': []},
             {'id': 'report_chunk_33.txt', 'score': 0.837199271, 'values': []},
             {'id': 'report_chunk_4.txt', 'score': 0.829370856, 'values': []},
             {'id': 'report_chunk_1.txt', 'score': 0.828416228, 'values': []},
             {'id': 'report_chunk_20.txt', 'score': 0.827207744, 'values': []}],
 'namespace': ''}